## Assignment 1 

#### Rating Prediction

In [1]:
import numpy
import urllib
import scipy.optimize
import random
import gzip
import csv
from collections import defaultdict
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [2]:
# input file using gzip
path = "train_Interactions.csv.gz"
path = "/Users/gakiara/Desktop/258R/assignment1/train_Interactions.csv.gz"
f = gzip.open(path, "rt", encoding="utf8")
reader = csv.reader(f, delimiter=",")

In [3]:
# reading the file to build datasets
datasets = []
first = True
for line in reader:
    if first:
        header = line
        first = False
    else:
        d = dict(zip(header, line))
        # convert strings to integers for some fields
        d["rating"] = int(d["rating"])
        datasets.append(d)

In [4]:
# split the training data
data_train = datasets[:190000]
data_valid = datasets[190000:]

In [5]:
# computer the global mean
globalAverage = sum([d["rating"] for d in data_train]) / len(data_train)
globalAverage
# calculate initial value of alpha,beta_user and beta_book
alpha = globalAverage
userRatings = defaultdict(list)
bookRatings = defaultdict(list)

UserPerBook = defaultdict(list)
BookPerUser = defaultdict(list)
Pair_Rating = defaultdict(int)

for l in data_train:
    user, book = l["userID"], l["bookID"]
    UserPerBook[book].append(user)
    BookPerUser[user].append(book)
    userRatings[user].append(l["rating"])
    bookRatings[book].append(l["rating"])
    Pair_Rating[(user,book)] = int(l["rating"])


# beta_user,beta_book,beta_pair
userBias = defaultdict(float)
for u in userRatings:
    userBias[u] = globalAverage - (sum(userRatings[u]) / len(userRatings[u]))

bookBias = defaultdict(float)
for b in bookRatings:
    bookBias[b] = globalAverage - (sum(bookRatings[b]) / len(bookRatings[b]))


In [6]:
print(len(userBias))
print(len(bookBias))

27917
6684


In [7]:
# define function for optimization
def differenciate(globalAverage, userBias, bookBias, lamb1,lamb2, epsilon):
    end = False
    globalAverage_last = 0
    userBias_last = userBias
    bookBias_last = bookBias
    MSE_last = 0
    cost_last = 0
    dataset = data_train
    iters = 0
    while not end:
        # update alpha
        globalAverage = 0
        for i in dataset:
            user, book = i["userID"], i["bookID"]
            globalAverage += i["rating"] - userBias_last[user] - bookBias_last[book]

        globalAverage = globalAverage / len(dataset)

        # update beta_user
        num_book = defaultdict(int)
        for u in userBias:
            userBias[u] = 0
        for i in dataset:
            user, book = i["userID"], i["bookID"]
            num_book[user] += 1
            userBias[user] += i["rating"] - globalAverage - bookBias_last[book]
        for u in userBias:
            userBias[u] = userBias[u] / (lamb1 + num_book[u])

        # update beta_book
        num_user = defaultdict(int)
        for b in bookBias:
            bookBias[b] = 0
        for i in dataset:
            user, book = i["userID"], i["bookID"]
            num_user[book] += 1
            bookBias[book] += i["rating"] - globalAverage - userBias[user]
        for b in bookBias:
            bookBias[b] = bookBias[b] / (lamb2 + num_user[b])

        predictions = []
        
        for d in dataset:
            user = d["userID"]
            book = d["bookID"]
            if user in userBias and book in bookBias:
                result = globalAverage + userBias[user] + bookBias[book]
            else:
                result = globalAverage
            predictions.append(result)
        
        labels = [l["rating"] for l in dataset]
        differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
        MSE = sum(differences) / len(differences)
        
        cost = MSE
        for u in userBias:
            cost += lamb1 * (userBias[u]**2)
        for i in bookBias:
            cost += lamb2 * (bookBias[i]**2)
        iters+=1;
        
        if (
            abs(MSE - MSE_last) < epsilon
            and abs(cost - cost_last) < epsilon
        ):
            end = True
            print(iters)
        else:
            globalAverage_last = globalAverage
            userBias_last = userBias
            bookBias_last = bookBias
            MSE_last = MSE
            cost_last = cost

    return globalAverage, userBias, bookBias

In [8]:
def getDirty0(Pair_Rating, BookPerUser, UserPerBook, dirty_feature):
    dirty_u = []
    dirty_i = []
    [dirty_limit, rating_std, rating_bound] = dirty_feature
    for u in BookPerUser.keys():
        if dirty_limit < len(BookPerUser[u]):
            rating_list = []
            for i in BookPerUser[u]:
                rating_list.append(Pair_Rating[(u,i)])
            if numpy.std(rating_list) < rating_std:
                if numpy.mean(rating_list) > rating_bound[1] or numpy.mean(rating_list) < rating_bound[0]:
                    dirty_u.append(u)
    for i in UserPerBook.keys():
        if dirty_limit < len(UserPerBook[i]):
            rating_set = set()
            for u in UserPerBook[i]:
                rating_set.add(Pair_Rating[(u,i)])
            if len(rating_set) == 1:
                dirty_i.append(i)
    return [dirty_u, dirty_i]

In [9]:
def getDirty(Pair_Rating, BookPerUser, UserPerBook, dirty_feature):
    dirty_u = []
    dirty_i = []
    #[dirty_limit, rating_std, rating_bound] = dirty_feature
    [dirty_limit, rating_std] = dirty_feature
    for u in BookPerUser.keys():
        # data too sparse
        if len(BookPerUser[u]) < dirty_limit[0]:
            dirty_u.append(u)
        else:
            rating_list = []
            for i in BookPerUser[u]:
                rating_list.append(Pair_Rating[(u,i)])
            #data is not stable
            if numpy.std(rating_list) > rating_std:
                dirty_u.append(u)

    for i in UserPerBook.keys():
        if len(UserPerBook[i]) < dirty_limit[1]:
            dirty_i.append(i)

    return [dirty_u, dirty_i]

In [10]:
def intial_clean0(dataset, dirtyfeature):
    dataset_clean = []
    [dirty_u, dirty_i] = getDirty0(Pair_Rating, BookPerUser, UserPerBook, dirtyfeature)
    for d in dataset:
        user, book = d["userID"], d["bookID"]
        if user not in dirty_u and book not in dirty_i:
            dataset_clean.append(d)

    # computer the global mean
    globalAverage = sum([d["rating"] for d in dataset_clean]) / len(dataset_clean)
    # calculate initial value of alpha,beta_user and beta_book
    alpha = globalAverage
    userRatings = defaultdict(list)
    bookRatings = defaultdict(list)

    for l in dataset_clean:
        user, book = l["userID"], l["bookID"]
        userRatings[user].append(l["rating"])
        bookRatings[book].append(l["rating"])


    # beta_user,beta_book,beta_pair
    userBias = defaultdict(float)
    for u in userRatings:
        userBias[u] = globalAverage - (sum(userRatings[u]) / len(userRatings[u]))

    bookBias = defaultdict(float)
    for b in bookRatings:
        bookBias[b] = globalAverage - (sum(bookRatings[b]) / len(bookRatings[b]))

    return dataset_clean,globalAverage, userBias, bookBias

In [11]:
def intial_clean(dataset, dirtyfeature):
    dataset_clean = []
    [dirty_u, dirty_i] = getDirty(Pair_Rating, BookPerUser, UserPerBook, dirtyfeature)
    for d in dataset:
        user, book = d["userID"], d["bookID"]
        if user not in dirty_u and book not in dirty_i:
            dataset_clean.append(d)

    # computer the global mean
    globalAverage = sum([d["rating"] for d in dataset_clean]) / len(dataset_clean)
    # calculate initial value of alpha,beta_user and beta_book
    alpha = globalAverage
    userRatings = defaultdict(list)
    bookRatings = defaultdict(list)

    for l in dataset_clean:
        user, book = l["userID"], l["bookID"]
        userRatings[user].append(l["rating"])
        bookRatings[book].append(l["rating"])


    # beta_user,beta_book,beta_pair
    userBias = defaultdict(float)
    for u in userRatings:
        userBias[u] = globalAverage - (sum(userRatings[u]) / len(userRatings[u]))

    bookBias = defaultdict(float)
    for b in bookRatings:
        bookBias[b] = globalAverage - (sum(bookRatings[b]) / len(bookRatings[b]))

    return dataset_clean,globalAverage, userBias, bookBias

In [12]:
# define function for optimization
def differenciate2(globalAverage, userBias, bookBias, lamb1,lamb2, epsilon, dataset):

    end = False
    globalAverage_last = 0
    userBias_last = userBias
    bookBias_last = bookBias
    MSE_last = 0
    cost_last = 0

    iters = 0
    while not end:
        # update alpha
        globalAverage = 0
        for i in dataset:
            user, book = i["userID"], i["bookID"]
            globalAverage += i["rating"] - userBias_last[user] - bookBias_last[book]

        globalAverage = globalAverage / len(dataset)

        # update beta_user
        num_book = defaultdict(int)
        for u in userBias:
            userBias[u] = 0
        for i in dataset:
            user, book = i["userID"], i["bookID"]
            num_book[user] += 1
            userBias[user] += i["rating"] - globalAverage - bookBias_last[book]
        for u in userBias:
            userBias[u] = userBias[u] / (lamb1 + num_book[u])

        # update beta_book
        num_user = defaultdict(int)
        for b in bookBias:
            bookBias[b] = 0
        for i in dataset:
            user, book = i["userID"], i["bookID"]
            num_user[book] += 1
            bookBias[book] += i["rating"] - globalAverage - userBias[user]
        for b in bookBias:
            bookBias[b] = bookBias[b] / (lamb2 + num_user[b])

        predictions = []

        for d in dataset:
            user = d["userID"]
            book = d["bookID"]
            if user in userBias and book in bookBias:
                result = globalAverage + userBias[user] + bookBias[book]
            else:
                result = globalAverage
            predictions.append(result)

        labels = [l["rating"] for l in dataset]
        differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
        MSE = sum(differences) / len(differences)

        cost = MSE
        for u in userBias:
            cost += lamb1 * (userBias[u]**2)
        for i in bookBias:
            cost += lamb2 * (bookBias[i]**2)
        iters+=1;

        if (
            abs(MSE - MSE_last) < epsilon
            and abs(cost - cost_last) < epsilon
        ):
            end = True
            print(iters)
        else:
            globalAverage_last = globalAverage
            userBias_last = userBias
            bookBias_last = bookBias
            MSE_last = MSE
            cost_last = cost

    return globalAverage, userBias, bookBias

In [34]:
# prediction result
def prediction(user, book):
    if user in userBias and book in bookBias:
        result = globalAverage_new + userBias_new[user] + bookBias_new[book]
    else:
        result = globalAverage_new
    if result < 0.75: result = 0
    if result > 0.75 and result < 1.25:
        if  result > 0.9 and result < 1.1:
            result = 1
        else:
            result = 0
    if result > 1.9 and result < 2.1: result = 2
    if result > 4.9: result = 5

    return result

In [14]:
# prediction result
def prediction2(user, book):
    if user in userBias and book in bookBias:
        result = globalAverage_new + userBias_new[user] + bookBias_new[book]
    elif user in userBias:
        result = globalAverage_new + userBias_new[user]
    elif book in bookBias:
        result = globalAverage_new + bookBias_new[book]
    else:
        result = globalAverage_new
    if result < 1: result = 0
    #if result > 5: result = 5
    return result

In [15]:
print(random.uniform(0,1))

0.8763505714701068


In [16]:
# define MSE function
def MSE(dataset):
    predictions = [prediction(d["userID"], d["bookID"]) for d in dataset]
    labels = [d["rating"] for d in dataset]
    differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
    mse = sum(differences) / len(differences)
    return mse

In [17]:
# define MSE function
def MSE2(dataset):
    predictions = [prediction2(d["userID"], d["bookID"]) for d in dataset]
    labels = [d["rating"] for d in dataset]
    differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
    mse = sum(differences) / len(differences)
    return mse

In [35]:
m = MSE(data_valid)
print(m)

1.3729677456136185


In [17]:
globalAverage_new, userBias_new, bookBias_new = differenciate(globalAverage, userBias, bookBias, 3.87,14.76, epsilon=1e-7)

83


In [344]:
print(globalAverage_new)

3.5666468524458494


In [345]:
m = MSE(data_valid)
print(m)

1.3729677456136185


In [ ]:
dirty_limits = [10,12,14,16,18,20]
sds = [0.05,0.08,0.1,0.2]
dirty_bound = [[1.5,4.5],[2,4],[2.6, 4.4],[2.5,3.5],[3,3]]
dirty_mse = defaultdict(list)
mse = []
for dl in dirty_limits:
    for s in sds:
        for db in dirty_bound:
             dirty_feature = [dl, s ,db]
             dataset_clean,globalAverage,userBias,userBias = intial_clean0(data_train,dirty_feature)
             globalAverage_new, userBias_new, bookBias_new = differenciate2(globalAverage, userBias, bookBias,3.87,14.76,1e-06,dataset_clean)
             m = MSE(data_valid)
             print(m,dl,s,db)
             print(len(dataset_clean))
             mse.append(m)
             dirty_mse[m] = dirty_feature

62
1.3876760126046812 10 0.05 [1.5, 4.5]
189134
63
1.3876760126090606 10 0.05 [2, 4]
189134
63
1.3876760126090606 10 0.05 [2.6, 4.4]
189134
63
1.3876759585110983 10 0.05 [2.5, 3.5]
189117
63
1.3876759585110612 10 0.05 [3, 3]
189117
63
1.3876760126090297 10 0.08 [1.5, 4.5]
189134


In [ ]:
best_mse = min(mse)
best_df = dirty_mse[best_mse]
print("Optimal dirty feature:", best_df, "; MSE:", best_mse)

In [ ]:
dirty_limits = [[2,2],[2,3],[2,4],[2,5]]
sds = [2.0,2.2,2.4,2.6,2.8,3,3.2]
#dirty_bound = [[2.6,4.4],[2.8,4.2],[3,4],[3.5,3.5]]
dirty_mse = defaultdict(list)
mse = []
for dl in dirty_limits:
    for s in sds:
        #for db in dirty_bound:
             dirty_feature = [dl, s ]
             dataset_clean,globalAverage,userBias,userBias = intial_clean(data_train,dirty_feature)
             globalAverage_new, userBias_new, bookBias_new = differenciate2(globalAverage, userBias, bookBias,3.87,14.67,1e-05,dataset_clean)
             m = MSE(data_valid)
             print(m,dl,s)
             print(len(dataset_clean))
             mse.append(m)
             dirty_mse[m] = dirty_feature

63
1.3936207409282109 [2, 2] 2.0
182813
53
1.379786736516174 [2, 2] 2.2
186703
53
1.3776701961835294 [2, 2] 2.4
188574
53
1.3740228906606438 [2, 2] 2.6
189191
53
1.374022890640953 [2, 2] 2.8
189191
53
1.374022890640953 [2, 2] 3
189191
53
1.374022890640953 [2, 2] 3.2
189191
57
1.3935437165072624 [2, 3] 2.0
182421
52
1.3797775476867986 [2, 3] 2.2
186300
53
1.377710443478119 [2, 3] 2.4
188168
52
1.3742149915497897 [2, 3] 2.6
188783
53
1.3742149915668351 [2, 3] 2.8
188783
53
1.3742149915668358 [2, 3] 3
188783
53
1.3742149915668358 [2, 3] 3.2
188783
58
1.395186316715228 [2, 4] 2.0
181477
53
1.3813734197033414 [2, 4] 2.2
185330


In [33]:
best_mse = min(mse)
best_df = dirty_mse[best_mse]
print("Optimal dirty feature:", best_df, "; MSE:", best_mse)

Optimal dirty feature: [[2, 3], 2.6] ; MSE: 1.3750365123654429


In [14]:
#Find the best lambda

In [11]:
lamb1 = [i for i in range(1,20)]
lamb2 = [i for i in range(1,20)]
mse = []
lambs_mse = defaultdict(list)
#when optimal the lamb1,epsilon should be bigger
for r1 in lamb1:
  for r2 in lamb2:
    globalAverage_new, userBias_new, bookBias_new = differenciate(
        globalAverage, userBias, bookBias, r1,r2, epsilon=1e-05
    )
    m = MSE(data_valid)
    lambs_mse[m]=[r1,r2]
    mse.append(m)

In [12]:
print(lambs_mse)

defaultdict(<class 'list'>, {1.434818876252579: [1, 1], 1.4267112938619115: [1, 2], 1.4218314528272191: [1, 3], 1.418618302489394: [1, 4], 1.4164029240662668: [1, 5], 1.414836660374625: [1, 6], 1.4137165175653645: [1, 7], 1.412915325369145: [1, 8], 1.4123490807479417: [1, 9], 1.4119600027678116: [1, 10], 1.4117070495901665: [1, 11], 1.411560295067007: [1, 12], 1.4114974364564854: [1, 13], 1.4115015427453337: [1, 14], 1.4115595557457516: [1, 15], 1.411661264861534: [1, 16], 1.411798590318674: [1, 17], 1.411965070639355: [1, 18], 1.4121554899099469: [1, 19], 1.4080327478215497: [2, 1], 1.4001421230018434: [2, 2], 1.3953549593067822: [2, 3], 1.3921799835769548: [2, 4], 1.3899749173391565: [2, 5], 1.388403580072648: [2, 6], 1.3872695144727214: [2, 7], 1.3864492305680354: [2, 8], 1.3858609027502569: [2, 9], 1.38544808446297: [2, 10], 1.3851705716412328: [2, 11], 1.3849989725128091: [2, 12], 1.384911327711484: [2, 13], 1.3848909264996667: [2, 14], 1.3849248502004614: [2, 15], 1.3850029741171

In [13]:
# Optimal λ
best_mse = min(mse)
best_lamda = lambs_mse[best_mse]
print("Optimal λ:", best_lamda, "; MSE:", best_mse)

Optimal λ: [4, 15] ; MSE: 1.3738393608002175


In [ ]:
#finetuning the lambda

In [58]:
# calculate initial value of alpha,beta_user and beta_book
# computer the global mean
# computer the global mean
globalAverage = sum([d["rating"] for d in data_train]) / len(data_train)
globalAverage

alpha = globalAverage

# beta_user,beta_book
userBias = defaultdict(float)
for u in userRatings:
    userBias[u] = globalAverage - (sum(userRatings[u]) / len(userRatings[u]))

bookBias = defaultdict(float)
for b in bookRatings:
    bookBias[b] = globalAverage - (sum(bookRatings[b]) / len(bookRatings[b]))

In [63]:
lamb1 = [i/100 for i in range(300,510)]
mse = []
lamb1_mse = defaultdict(list)
#when optimal the lamb1,epsilon should be bigger
for r1 in lamb1:
    globalAverage_new, userBias_new, bookBias_new = differenciate(
        globalAverage, userBias, bookBias, r1,15, epsilon=1e-05
    )
    m = MSE(data_valid)
    lamb1_mse[m]=[r1]
    mse.append(m)

49
28
28
28
28
28
28
28
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25


In [64]:
print(lamb1_mse)

defaultdict(<class 'list'>, {1.3756145323055027: [3.0], 1.3755686439449872: [3.01], 1.3755234635205915: [3.02], 1.3754789863287604: [3.03], 1.3754352072309444: [3.04], 1.3753921211321811: [3.05], 1.375349722981196: [3.06], 1.3753080077698878: [3.07], 1.3752669705965097: [3.08], 1.3752266064105576: [3.09], 1.3751869103945336: [3.1], 1.3751478777087076: [3.11], 1.3751095035542196: [3.12], 1.3750717831727552: [3.13], 1.375034711846145: [3.14], 1.3749982848958815: [3.15], 1.3749624976827852: [3.16], 1.3749273456065425: [3.17], 1.3748928241053435: [3.18], 1.3748589286554431: [3.19], 1.374825654770803: [3.2], 1.3747929980027176: [3.21], 1.3747609539393746: [3.22], 1.3747295182055443: [3.23], 1.3746986864621489: [3.24], 1.3746684544059444: [3.25], 1.3746388177691378: [3.26], 1.3746097723190323: [3.27], 1.374581313857639: [3.28], 1.3745534382213838: [3.29], 1.374526141280733: [3.3], 1.374499418939846: [3.31], 1.3744732671362545: [3.32], 1.37444768184053: [3.33], 1.3744226590559077: [3.34], 1.3

In [65]:
# Optimal λ
best_mse = min(mse)
best_lamda1 = lamb1_mse[best_mse]
print("Optimal λ1:", best_lamda1, "; MSE:", best_mse)

Optimal λ1: [3.87] ; MSE: 1.373807330567233


In [66]:
# calculate initial value of alpha,beta_user and beta_book
# computer the global mean
# computer the global mean
globalAverage = sum([d["rating"] for d in data_train]) / len(data_train)
globalAverage

alpha = globalAverage

# beta_user,beta_book
userBias = defaultdict(float)
for u in userRatings:
    userBias[u] = globalAverage - (sum(userRatings[u]) / len(userRatings[u]))

bookBias = defaultdict(float)
for b in bookRatings:
    bookBias[b] = globalAverage - (sum(bookRatings[b]) / len(bookRatings[b]))

In [67]:
lamb2 = [i/100 for i in range(1450,1510)]
mse = []
lamb2_mse = defaultdict(list)
#when optimal the lamb1,epsilon should be bigger
for r2 in lamb2:
    globalAverage_new, userBias_new, bookBias_new = differenciate(
        globalAverage, userBias, bookBias, 3.87, r2, epsilon=1e-05
    )
    m = MSE(data_valid)
    lamb2_mse[m]=[r2]
    mse.append(m)

66
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29


In [68]:
print(lamb2_mse)

defaultdict(<class 'list'>, {1.373807510382738: [14.5], 1.3738073847405632: [14.51], 1.3738072646768218: [14.52], 1.3738071497249675: [14.53], 1.373807039874539: [14.54], 1.3738069351153697: [14.55], 1.3738068354373298: [14.56], 1.3738067408302994: [14.57], 1.3738066512841751: [14.58], 1.3738065667889046: [14.59], 1.3738064873344247: [14.6], 1.3738064129107206: [14.61], 1.3738063435077963: [14.62], 1.3738062791156729: [14.63], 1.3738062197243948: [14.64], 1.3738061653240408: [14.65], 1.37380611590469: [14.66], 1.3738060714564768: [14.67], 1.3738060319695198: [14.68], 1.3738059973722718: [14.69], 1.3738059677784706: [14.7], 1.37380594311652: [14.71], 1.3738059233766238: [14.72], 1.373805908549032: [14.73], 1.3738058986240058: [14.74], 1.373805893591847: [14.75], 1.3738058934428816: [14.76], 1.373805898167447: [14.77], 1.3738059077559042: [14.78], 1.3738059221986412: [14.79], 1.373805941486058: [14.8], 1.3738059656085977: [14.81], 1.3738059945567123: [14.82], 1.3738060283208535: [14.83],

In [69]:
# Optimal λ
best_mse = min(mse)
best_lamda2 = lamb2_mse[best_mse]
print("Optimal λ2:", best_lamda2, "; MSE:", best_mse)

Optimal λ1: [14.76] ; MSE: 1.3738058934428816


In [ ]:
#finetuning the epsilon

In [70]:
mse = []
epsilon_mse = defaultdict()
epsilons = [10**(-i) for i in range(6,15)]
for epsilon in epsilons:
    globalAverage_new, userBias_new, bookBias_new = differenciate(
        globalAverage, userBias, bookBias, 3.87,14.76, epsilon
    )
    m = MSE(data_valid)
    epsilon_mse[m]=epsilon
    mse.append(m)

53
9
9
9
8
7
8
2
2


In [71]:
print(epsilon_mse)

defaultdict(None, {1.3738058937407898: 1e-06, 1.37380589372046: 1e-07, 1.3738058937183002: 1e-08, 1.3738058937180793: 1e-09, 1.3738058937180517: 1e-10, 1.373805893718053: 1e-11, 1.3738058937180533: 1e-12, 1.3738058937180537: 1e-14})


In [72]:
# Optimal λ
best_mse = min(mse)
best_epsilon = epsilon_mse[best_mse]
print("Optimal epsilon:", best_epsilon, "; MSE:", best_mse)

Optimal epsilon: 1e-10 ; MSE: 1.3738058937180517


Optimal λ3: [3, 14, 13] ; MSE: 1.3744219296698006


#### USE ALL the DATA

In [36]:
# computer the global mean
globalAverage = sum([d["rating"] for d in datasets]) / len(datasets)
globalAverage

3.687005

In [37]:
# calculate initial value of alpha,beta_user and beta_book
alpha = globalAverage
userRatings = defaultdict(list)
bookRatings = defaultdict(list)

for l in datasets:
    user, book = l["userID"], l["bookID"]
    userRatings[user].append(l["rating"])
    bookRatings[book].append(l["rating"])

# beta_user,beta_book
userBias = defaultdict(float)
for u in userRatings:
    userBias[u] = globalAverage - (sum(userRatings[u]) / len(userRatings[u]))

bookBias = defaultdict(float)
for b in bookRatings:
    bookBias[b] = globalAverage - (sum(bookRatings[b]) / len(bookRatings[b]))

In [38]:
# define function for optimization
def differenciate(globalAverage, userBias, bookBias, lamb1,lamb2, epsilon):
    end = False
    globalAverage_last = 0
    userBias_last = userBias
    bookBias_last = bookBias
    MSE_last = 0
    cost_last = 0
    dataset = datasets
    while not end:
        # update alpha
        globalAverage = 0
        for i in dataset:
            user, book = i["userID"], i["bookID"]
            globalAverage += i["rating"] - userBias_last[user] - bookBias_last[book]

        globalAverage = globalAverage / len(dataset)

        # update beta_user
        num_book = defaultdict(int)
        for u in userBias:
            userBias[u] = 0
        for i in dataset:
            user, book = i["userID"], i["bookID"]
            num_book[user] += 1
            userBias[user] += i["rating"] - globalAverage - bookBias_last[book]
        for u in userBias:
            userBias[u] = userBias[u] / (lamb1 + num_book[u])

        # update beta_book
        num_user = defaultdict(int)
        for b in bookBias:
            bookBias[b] = 0
        for i in dataset:
            user, book = i["userID"], i["bookID"]
            num_user[book] += 1
            bookBias[book] += i["rating"] - globalAverage - userBias[user]
        for b in bookBias:
            bookBias[b] = bookBias[b] / (lamb2 + num_user[b])

        predictions = []

        for d in dataset:
            user = d["userID"]
            book = d["bookID"]
            if user in userBias and book in bookBias:
                result = globalAverage + userBias[user] + bookBias[book]
            else:
                result = globalAverage
            predictions.append(result)

        labels = [l["rating"] for l in dataset]
        differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
        MSE = sum(differences) / len(differences)

        cost = MSE
        for u in userBias:
            cost += lamb1 * (userBias[u]**2)
        for i in bookBias:
            cost += lamb2 * (bookBias[i]**2)


        if (
            abs(MSE - MSE_last) < epsilon
            and abs(cost - cost_last) < epsilon
        ):
            end = True
        else:
            globalAverage_last = globalAverage
            userBias_last = userBias
            bookBias_last = bookBias
            MSE_last = MSE
            cost_last = cost

    return globalAverage, userBias, bookBias

In [39]:

globalAverage_new, userBias_new, bookBias_new = differenciate(
    globalAverage, userBias, bookBias, 3.87,14.76, epsilon=1e-7)
'''
globalAverage_new, userBias_new, bookBias_new, pairBias_new = differenciate2(globalAverage, userBias, bookBias, pairBias, 3.853, 14.74, 600, epsilon=1e-07)
'''

In [41]:
predictions = open("/Users/gakiara/Desktop/258R/assignment1/predictions_Rating.csv", "w")
for l in open("/Users/gakiara/Desktop/258R/assignment1/pairs_Rating.csv"):
    if l.startswith("userID"):
        # header
        predictions.write(l)
        continue
    u, b = l.strip().split(",")
    p = prediction(u, b)
    predictions.write(u + "," + b + "," + str(p) + "\n")
predictions.close()
#1.4547551286